In [1]:
pwd

'C:\\Users\\vigne_u98gpxi\\Desktop\\Lavviggy\\Lav\\HuntJob\\Income\\DE_CaseStudy_Dataset\\Sales_Data'

In [2]:
import sys
print(sys.version)
print(sys.exec_prefix)

3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]
C:\Users\vigne_u98gpxi\anaconda3


In [3]:
import pyspark
from pyspark.sql import SparkSession
print(pyspark.__version__)

3.1.1


In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.functions import avg, round, col,sum,count
#import org.apache.spark.sql.functions.col
#import org.apache.spark.sql.types.IntegerType

def create_spark_session(app_name: str) -> SparkSession:
    """ Create a spark session.
    """
#     ss = SparkSession.builder.appName(app_name).getOrCreate()#.master('local').appName(app_name).getOrCreate()
#     ss = SparkSession.builder.appName(app_name).config("spark.jars", "mysql-connector-j-8.3.0.jar").getOrCreate()#.master('local').appName(app_name).getOrCreate()


#     ss = SparkSession.builder.appName(app_name).config("spark.jars", "spark-mssql-connector_2.12-1.2.0.jar").getOrCreate()#.master('local').appName(app_name).getOrCreate()
#     ss = SparkSession.builder.appName(app_name).config("spark.jars", "mssql-jdbc-12.6.1.jre11.jar").getOrCreate()#.master('local').appName(app_name).getOrCreate()    
#     ss = SparkSession.builder.appName(app_name).config("spark.jars", "spark-mssql-connector_2.12-1.4.0-BETA.jar").getOrCreate()#.master('local').appName(app_name).getOrCreate()

#     ss = SparkSession.builder.appName(app_name).config("spark.jars", "spark-mssql-connector_2.12-1.3.0-BETA.jar").getOrCreate()#.master('local').appName(app_name).getOrCreate()
#     ss = SparkSession.builder.appName(app_name).config("spark.jars", "spark-mssql-connector_2.12-1.3.0-BETA").getOrCreate()#.master('local').appName(app_name).getOrCreate()

#     ss = SparkSession.builder.appName(app_name).config("spark.jars.packages", "com.microsoft.azure:spark-mssql-connector_2.12:1.3.0").getOrCreate()#.master('local').appName(app_name).getOrCreate()

    ss = SparkSession.builder.appName(app_name).config("spark.jars", "spark-mssql-connector_2.12-1.2.0.jar").getOrCreate()#.master('local').appName(app_name).getOrCreate()

#     ss = SparkSession.builder.appName(app_name).config("spark.jars.packages", "com.microsoft.azure:spark-mssql-connector_2.12:1.2.0").getOrCreate()#.master('local').appName(app_name).getOrCreate()

    

    return ss


def read_in_data(sc: SparkSession, file: str):
    """ Return a spark DataFrame of the excel file <file>.
    """
    print(file)
    return sc.read.csv(file, header='true', sep=',', inferSchema=True)


def sum_by_feature(df: DataFrame, feature: str) -> DataFrame:
    """ Find the average salary of <feature> by groups.
    """
   
    return df.groupby(feature) \
        .agg(sum('Price').alias('Total_Price')) \
        .sort('Total_Price', ascending=False)

def count_by_feature(df: DataFrame, feature: str) -> DataFrame:
    """ Find the average salary of <feature> by groups.
    """
    df.printSchema()
    return df.groupby(feature) \
        .agg(count('Price').alias('Total_Quantity')) \
        .sort('Total_Quantity', ascending=False)


def output_result(df: DataFrame, output_location: str, output_folder: str) -> None:
    """ Save the DataFrame <df> as a csv file in the location specified by
    <output_location>.
    """
    # condense all data points in one single file
    df.coalesce(1).write.csv(path=output_location + output_folder,
                             mode='append', header=True)


if __name__ == '__main__':

    # create a spark session
    spark = create_spark_session("average salary")

    #### EXTRACT ####

    # read in the train_features.csv as a spark DataFrame
    books_df = read_in_data(spark, 'books_data.csv')

    # read in the train_salaries.csv as a spark DataFrame
    books_ratings_df = read_in_data(spark, 'Books_rating.csv')
    

    #### TRANSFORM ####


    df=books_ratings_df.withColumn('Price',col('Price').cast("integer"))
    print(df.printSchema())
  
    sum_price_by_title = sum_by_feature(df, 'Title')
    count_price_by_title = count_by_feature(df, 'Title')
    

    #### LOAD ####

    #OUTPUT_LOCATION = '/output'

    # save the above results as csv files in the bucket
#     output_result(sum_price_by_title, OUTPUT_LOCATION, 'Title')
#     output_result(count_price_by_title, OUTPUT_LOCATION, 'Title')
  

    spark.stop()

books_data.csv
Books_rating.csv
root
 |-- Id: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- User_id: string (nullable = true)
 |-- profileName: string (nullable = true)
 |-- review/helpfulness: string (nullable = true)
 |-- review/score: string (nullable = true)
 |-- review/time: string (nullable = true)
 |-- review/summary: string (nullable = true)
 |-- review/text: string (nullable = true)

None
root
 |-- Id: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- User_id: string (nullable = true)
 |-- profileName: string (nullable = true)
 |-- review/helpfulness: string (nullable = true)
 |-- review/score: string (nullable = true)
 |-- review/time: string (nullable = true)
 |-- review/summary: string (nullable = true)
 |-- review/text: string (nullable = true)



In [5]:
# books_df.printSchema()

In [6]:
# CREATE TABLE books_data (
#     Title String,
#     description String,
#     authors String,
#     image String
#     previewLink String
#     publisher String
#     publishedDate String
#     infoLink String
#     categories String
#     ratingsCount String
   
# );

In [7]:
# books_df.limit(2).write \
#     .format("jdbc") \
#     .option("driver","com.mysql.cj.jdbc.Driver") \
#     .option("url", "jdbc:mysql://income-sales-db.cjuyq4k026ji.us-east-2.rds.amazonaws.com:1433/income") \
#     .option("dbtable", "sales.books_data") \
#     .option("user", "admin") \
#     .option("password", "Logindatabase24") \
#     .save()

In [8]:
# SERVER_ADDR_port = "income-sales-db.cjuyq4k026ji.us-east-2.rds.amazonaws.com:1433"
# db_name = "income"

# books_df.limit(2).write \
#     .format("jdbc") \
#     .mode("overwrite") \
#     .option("driver" , "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
#     .option("url", f"jdbc:sqlserver://{SERVER_ADDR_port};databaseName={db_name};integratedSecurity=true;encrypt=true;trustServerCertificate=true;") \
#     .option("dbtable", "sales.books_data") \
#     .option("user", "admin") \
#     .option("password", "Logindatabase24") \
#     .save()


# #     .format("com.microsoft.sqlserver.jdbc.spark") \
# #     .option("driver" , "com.microsoft.sqlserver.jdbc.SQLServerDriver") \


# #This driver is not configured for integrated authentication.

In [9]:
# SERVER_ADDR_port = "income-sales-db.cjuyq4k026ji.us-east-2.rds.amazonaws.com:1433"
# # SERVER_ADDR_port = "income-sales-db.cjuyq4k026ji.us-east-2.rds.amazonaws.com"

# db_name = "income"

# books_df.limit(2).write \
#     .format("com.microsoft.sqlserver.jdbc.SQLServerDriver") \
#     .option("url",f"jdbc:sqlserver://{SERVER_ADDR_port};databaseName={db_name};") \
#     .option("dbtable", "sales.books_data") \
#     .option("user", "admin") \
#     .option("password", "Logindatabase24") \
#     .save()

#com.microsoft.sqlserver.jdbc.SQLServerDriver does not allow create table as select.

In [10]:
# SERVER_ADDR_port = "income-sales-db.cjuyq4k026ji.us-east-2.rds.amazonaws.com:1433"
# # SERVER_ADDR_port = "income-sales-db.cjuyq4k026ji.us-east-2.rds.amazonaws.com"

# db_name = "income"

# books_df.limit(2).write \
#     .format("com.microsoft.sqlserver.jdbc.spark") \
#     .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
#     .mode("overwrite") \
#     .option("url",f"jdbc:sqlserver://{SERVER_ADDR_port};databaseName={db_name};") \
#     .option("dbtable", "sales.books_data") \
#     .option("user", "admin") \
#     .option("password", "Logindatabase24") \
#     .save()

# #java.lang.ClassNotFoundException: com.microsoft.sqlserver.jdbc.SQLServerDriver

In [11]:
# SERVER_ADDR_port = "income-sales-db.cjuyq4k026ji.us-east-2.rds.amazonaws.com:1433"
SERVER_ADDR_port = "income-sales-db.cjuyq4k026ji.us-east-2.rds.amazonaws.com"
db_name = "income"

books_df.limit(2).write \
    .format("com.microsoft.sqlserver.jdbc.spark") \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .mode("overwrite") \
    .option("url",f"jdbc:sqlserver://{SERVER_ADDR_port};databaseName={db_name};") \
    .option("dbtable", "sales.books_data") \
    .option("user", "admin") \
    .option("password", "Logindatabase24") \
    .save()

#java.lang.ClassNotFoundException: com.microsoft.sqlserver.jdbc.SQLServerDriver


Py4JJavaError: An error occurred while calling o88.save.
: java.lang.ClassNotFoundException: com.microsoft.sqlserver.jdbc.SQLServerDriver
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:476)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:588)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:521)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:102)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:102)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:102)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:215)
	at com.microsoft.sqlserver.jdbc.spark.SQLServerBulkJdbcOptions.<init>(SQLS